# NSRR Spectral Power

In [ ]:
import os
import glob
import yasa
import warnings
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from mne.io import read_raw_edf
from mne.filter import filter_data

from helper_functions import get_sub_visit_hyp

# Define paths
root_dir = '/Volumes/NSRR/'

In [ ]:
df = []
sf = 100

for dataset in ['mesa', 'cfs']:
    
    # Extract all EDF files
    all_edfs, hypno_dir = get_all_edfs(dataset, root_dir)
    
    # Include
    if dataset == "mesa":
        include = ["EEG3"]
    elif dataset == "cfs":
        include = ["C3"]

    for eeg_file in tqdm(all_edfs):

        #################################
        # DATA LOADING
        #################################
        
        # Extract subject, visit and hypno_file from fname
        sub, visit, hypno_file = get_sub_visit_hyp(eeg_file, dataset, hypno_dir)

        # Check that file exists
        if not os.path.isfile(eeg_file):
            warnings.warn("File not found %s" % eeg_file)
            continue
        if not os.path.isfile(hypno_file):
            warnings.warn("File not found %s" % hypno_file)
            continue

        # LOAD EEG DATA
        try:
            raw = read_raw_edf(eeg_file, preload=False, verbose=0)
            raw.drop_channels(np.setdiff1d(raw.info['ch_names'], include))
            raw.load_data()
        except:
            continue

        # Resample to 100 Hz and bandpass filter
        raw.resample(sf, npad="auto")
        raw.filter(0.5, 30)

        # LOAD HYPNOGRAM
        hypno, _ = yasa.load_profusion_hypno(hypno_file)
        if not len(hypno[hypno > 0]):
            continue

        # Upsample hypno
        sf_hyp = 1 / 30
        hypno_up = yasa.hypno_upsample_to_data(hypno, sf_hyp, raw)
        hypno_NREM = pd.Series(hypno).replace({2: 6, 3: 6}).to_numpy()
        hypno_NREM_up = yasa.hypno_upsample_to_data(hypno_NREM, sf_hyp, raw)

        # Extract C4-M1 data 
        # http://zzz.bwh.harvard.edu/luna/vignettes/nsrr-polarity/
        data = (raw.get_data()[0]) * 1e6
        data *= -1

        #################################
        # SPECTRAL POWERS
        #################################

        bands = [
            (0.5, 1.25, 'sdelta'),  # sdelta is 1.25 Hz (max SO frequency)
            (1.25, 4, 'fdelta'), 
            (4, 8, 'theta'),
            (8, 12, 'alpha'),
            (12, 16, 'sigma'),
            (16, 30, 'beta')  # Do not include gamma because of bandpass filter
        ]

        bp = yasa.bandpower(data, sf, ch_names=["EEG"], hypno=hypno_up, include=(2, 3, 4), bands=bands)
        bp_NREM = yasa.bandpower(data, sf, ch_names=["EEG"], hypno=hypno_NREM_up, include=(6), bands=bands)
        bp = bp.append(bp_NREM)
        bp.rename(columns={"TotalAbsPow": "total_pow"}, inplace=True)
        bp = bp.add_prefix("bp_")
        bp['subj'] = sub
        bp['dataset'] = dataset
        bp['visit'] = visit
        bp = bp.reset_index()
        bp.columns = bp.columns.str.lower()
        bp.drop(columns=['bp_freqres', 'bp_relative'], inplace=True)
        bp['stage'] = bp['stage'].replace({2: "N2", 3: "N3", 4: "REM", 6: "NREM"})
        bp = bp.pivot_table(index=['dataset', 'subj', 'visit'], columns='stage')
        bp.columns = ['_'.join(col).strip() for col in bp.columns.values]
    
        df.append(bp)

In [ ]:
df = pd.concat(df)
df.round(3)

In [ ]:
df.to_csv("output/csv/df_spectral.csv")